In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import os

import numpy as np
np.random.seed(2016)
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import list_pictures, array_to_img

from image_ext import list_pictures_in_multidir, load_imgs_asarray   

Using TensorFlow backend.


In [3]:
def create_fcn(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv5)

    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv5])
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv7), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(input=inputs, output=conv12)

    return fcn

In [4]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2.*intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)

In [5]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [8]:
if __name__ == '__main__':

    fpaths_xs_train = list_pictures_in_multidir(['.\\dataset\\train\\image\\'])
    fpaths_ys_train = list_pictures_in_multidir(['.\\dataset\\train\\gt\\'])
    fpaths_xs_valid = list_pictures_in_multidir(['.\\dataset\\valid\\image\\'])
    fpaths_ys_valid = list_pictures_in_multidir(['.\\dataset\\valid\\gt\\'])
    
    #dim_ordering ='tf' 
    dim_ordering = 'channels_first'
    
    target_size = (224, 224)
    
    # データを配列として取得
    print('loading data...')
    X_train = load_imgs_asarray(fpaths_xs_train, grayscale=False, 
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_train = load_imgs_asarray(fpaths_ys_train, grayscale=True, 
                                target_size=target_size, dim_ordering=dim_ordering)
    X_valid = load_imgs_asarray(fpaths_xs_valid, grayscale=False,
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_valid = load_imgs_asarray(fpaths_ys_valid, grayscale=True,
                                target_size=target_size, dim_ordering=dim_ordering)
    print('==> ' + str(len(X_train)) + ' training images loaded')
    print('==> ' + str(len(Y_train)) + ' training masks loaded')
    print('==> ' + str(len(X_valid)) + ' validation images loaded')
    print('==> ' + str(len(Y_valid)) + ' validation masks loaded')    

loading data...
==> 71 training images loaded
==> 71 training masks loaded
==> 12 validation images loaded
==> 12 validation masks loaded


In [9]:
    # 前処理
    print('computing mean and standard deviation...')
    mean = np.mean(X_train, axis=(0, 2, 3))
    std = np.std(X_train, axis=(0, 2, 3))
    print('==> mean: ' + str(mean))
    print('==> std : ' + str(std))
    
    fname_stats = 'stats.npy'

    print('saving mean and standard deviation to ' + fname_stats + '...')
    stats = {'mean': mean, 'std': std}
    np.savez(fname_stats, **stats)
    print('==> done')

    print('globally normalizing data...')
    for i in range(3):
        X_train[:, i] = (X_train[:, i] - mean[i]) / std[i]
        X_valid[:, i] = (X_valid[:, i] - mean[i]) / std[i]
    Y_train /= 255
    Y_valid /= 255
    print('==> done')

computing mean and standard deviation...
==> mean: [ 212.14715576  173.72639465  191.51356506]
==> std : [ 59.49325943  58.38890076  59.46165466]
saving mean and standard deviation to stats.npy...
==> done
globally normalizing data...
==> done


In [10]:
    # モデルを作成
    print('creating model...')
    model = create_fcn(target_size)
    model.summary()

creating model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 224, 224)  896         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 224, 224)  9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 32, 112, 112)  0           conv2d_2[0][0]                   
_________________________________________________________________________

c:\users\nakazawa atsushi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [12]:
# 損失関数，最適化手法を定義
adam = Adam(lr=1e-5)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

# 構造・重みを保存するディレクトリーの有無を確認
dpath_checkpoints = '.\\checkpoints\\'
if not os.path.isdir(dpath_checkpoints):
    os.mkdir(dpath_checkpoints)

In [17]:
# モデルの構造を保存
json_string = model.to_json()
fname_architecture = 'fcn.json'
fpath_architecture = os.path.join(dpath_checkpoints, fname_architecture)
with open(fpath_architecture, 'w', encoding='utf8') as f:
    f.write(json_string)

In [ ]:
# 重みを保存するためのオブジェクトを用意
fname_weights = 'weight'
fpath_weights = os.path.join(dpath_checkpoints, fname_weights)
checkpointer = ModelCheckpoint(filepath=fpath_weights, save_best_only=False)

# トレーニングを開始
print('start training...')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=20, verbose=1,
          shuffle=True, validation_data=(X_valid, Y_valid),
          callbacks=[checkpointer])
    
"""
else: # test
    # コマンドライン引数の正否をチェック
    assert(os.path.isfile(args.weights))

    # データを配列として取得
    print('loading data...')
    dpath_xs_test = os.path.join(dpath_data, 'test')
    fpaths_xs_test = list_pictures(dpath_xs_test)
    fnames_xs_test = [os.path.basename(fpath) for fpath in fpaths_xs_test]
    X_test = load_imgs_asarray(fpaths_xs_test, grayscale=False,
                               target_size=target_size,
                               dim_ordering=dim_ordering)
    print('==> ' + str(len(X_test)) +  ' test images loaded')

    # トレーニング時に計算した平均・標準偏差をロード
    print('loading mean and standard deviation from ' + fname_stats + '...')
    stats = np.load(fname_stats)
    mean = stats['mean']
    std = stats['std']
    print('==> mean: ' + str(mean))
    print('==> std : ' + str(std))

    print('globally normalizing data...')
    for i in range(3):
        X_test[:, i] = (X_test[:, i] - mean[i]) / std[i]
    print('==> done')

    # モデルを作成
    # （model_from_json()を使って保存してある構造を読み込むことも可能）
    print('creating model...')
    model = create_fcn(target_size)
    model.summary()

    # 学習済みの重みをロード
    fpath_weights = os.path.realpath(args.weights)
    print('loading weights from ' + fpath_weights)
    model.load_weights(fpath_weights)
    print('==> done')

    # テストを開始
    outputs = model.predict(X_test)

    # 出力を画像として保存
    dpath_outputs = os.path.join(dpath_this, dname_outputs)
    if not os.path.isdir(dpath_outputs):
        os.mkdir(dpath_outputs)

    print('saving outputs as images...')
    for i, array in enumerate(outputs):
        array = np.where(array > 0.5, 1, 0) # 二値に変換
        array = array.astype(np.float32)
        img_out = array_to_img(array, dim_ordering)
        fpath_out = os.path.join(dpath_outputs, fnames_xs_test[i])
        img_out.save(fpath_out)
    print('==> done')
"""

start training...


c:\users\nakazawa atsushi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 71 samples, validate on 12 samples
Epoch 1/20
